In [1]:
import requests
import json

In [8]:
#Get EcoBee application PIN and tokens

APP_KEY = 'API-KEY'
SCOPE = 'smartWrite'

auth_url = 'https://api.ecobee.com/authorize?'
payload = {'response_type':'ecobeePin','client_id': APP_KEY,'scope' : SCOPE}

r = requests.get(auth_url,payload)

r_dict = json.loads(r.text)

print('Your PIN is: ',r_dict['ecobeePin'])

('Your PIN is: ', u'3gxe')


In [9]:
#Get original tokens after PIN validation

import requests
import json

#POST https://api.ecobee.com/token?
#        grant_type=ecobeePin&
#        code=AUTHORIZATION_TOKEN&
#        client_id=APP_KEY

auth_code = r_dict['code']
token_url = 'https://api.ecobee.com/token?'
payload = {'grant_type':'ecobeePin','code': auth_code,'client_id': APP_KEY}

r_tokens = requests.post(token_url,params=payload)

r_tokens_dict = json.loads(r_tokens.text)

#print(r_tokens_dict)

access_token = r_tokens_dict['access_token']
token_type = r_tokens_dict['token_type']
refresh_token = r_tokens_dict['refresh_token']

print(access_token, token_type, refresh_token)

#Write tokens to file

f=open("tokens.txt","w+")
f.truncate()
f.write(access_token)
f.write(",")
f.write(token_type)
f.write(",")
f.write(refresh_token)
f.close

(u'uw90JUvQ23DdwPjbg1bXelxHuqUHkfdF', u'Bearer', u'uxzgzYVWN0ILEmg4goD3OkzrsazkwrRU')


<function close>

In [10]:
#Manually exchange tokens

token_url='https://api.ecobee.com/token?'
payload = {'grant_type':'refresh_token','refresh_token': refresh_token,'client_id': APP_KEY}
    
r_refresh = requests.post(token_url,params=payload)
    
r_refresh_dict = json.loads(r_refresh.text)

print(r_refresh_dict)
    
if r_refresh.status_code == requests.codes.ok:

    token_type = r_refresh_dict['token_type']
    access_token = r_refresh_dict['access_token']
    refresh_token = r_refresh_dict['refresh_token']

    auth = (token_type)+' '+(access_token)
    
    print(r_refresh.status_code)

# write tokens to file
    
    f=open("tokens.txt","w+")
    f.truncate()
    f.write(access_token)
    f.write(",")
    f.write(token_type)
    f.write(",")
    f.write(refresh_token)

# verify tokens are good

    token_str = str(f.read())
    f.close
    
    print(token_str)
    
#print("authentication error")
    

{u'access_token': u'Ok8OQ0DdVikDmKbivokG7kpv7SBWUV0o', u'token_type': u'Bearer', u'expires_in': 3599, u'refresh_token': u'CsbNAdTkHM6EBQPBn70MxlASz8WkhnTK', u'scope': u'smartWrite'}
200



In [11]:
# Check contents of token file if needed

f=open("tokens.txt","r")

token_str = str(f.read())

f.close

print(token_str)

token_list = token_str.split(",")

access_token = token_list[0]
token_type = token_list[1]
refresh_token = token_list[2]

print(access_token, token_type, refresh_token)

Ok8OQ0DdVikDmKbivokG7kpv7SBWUV0o,Bearer,CsbNAdTkHM6EBQPBn70MxlASz8WkhnTK
('Ok8OQ0DdVikDmKbivokG7kpv7SBWUV0o', 'Bearer', 'CsbNAdTkHM6EBQPBn70MxlASz8WkhnTK')


In [ ]:
#Main Program
# Sometimes hangs and throws exceptions, need to sort this out
# Separately loop and test Ecobee token exchange and switch operation
# Need to fix and test logic across full range.

# set up libraries

import requests
import json
from time import sleep
import datetime
from pyHS100 import SmartPlug
from pprint import pformat as pf

#initialize plug states before loop

office_plug_on = False 
mbed_plug_on = False 
orm_plug_on = False 
lrm_plug_on = False 

# Start log file

l=open("log.txt","a+")

# Start loop including time delay ...

while True:

# 1)  Read tokens from file

    f=open("tokens.txt","r")

    token_str = str(f.read())

    f.close

    token_list = token_str.split(",")

    access_token = token_list[0]
    token_type = token_list[1]
    refresh_token = token_list[2]

# 2) Submit old refresh token for valid access token and new refresh token

    token_url='https://api.ecobee.com/token?'
    payload = {'grant_type':'refresh_token','refresh_token': refresh_token,'client_id': APP_KEY}
    
    r_refresh = requests.post(token_url,params=payload)
    
    r_refresh_dict = json.loads(r_refresh.text)
    
    if r_refresh.status_code == requests.codes.ok:

        token_type = r_refresh_dict['token_type']
        access_token = r_refresh_dict['access_token']
        refresh_token = r_refresh_dict['refresh_token']

        auth = (token_type)+' '+(access_token)
    
#        print(r_refresh.status_code)

    #print(auth)

# 3) Write new tokens back to file

    f=open("tokens.txt","w+")
    f.truncate()
    f.write(access_token)
    f.write(",")
    f.write(token_type)
    f.write(",")
    f.write(refresh_token)

# 4)  Call EcoBee API to get data

    thermostat_url = 'https://api.ecobee.com/1/thermostat?'
    payload = {'json': ('{"selection":{"selectionType":"registered","includeSensors":"true"}}')}
    header = {'Content-Type': 'application/json;charset=UTF-8','Authorization': auth}

    request = requests.get(thermostat_url,headers=header,params=payload)

    thermostat_response = request.json()['thermostatList']

#print(thermostat_response) #test

# 5) Parse response into variables

    tstat_string = str(thermostat_response)

    tstat_list = tstat_string.split(",")

    tstat_tuple = tuple(tstat_list)

    tstat_name = tstat_tuple[26][12:-3]
#print(tstat_name) #test

    office_temp = float(tstat_tuple[16][13:-2])/10
#print(office_temp) #test

    mbed_temp = float(tstat_tuple[8][13:-2])/10
#print(mbed_temp)

# 5)  Decision logic and set booleans for switches

#timestamp and time for logic

    timestamp = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())

    dectime = float(timestamp[11:13])+float(timestamp[14:16])/60
    
# Setpoints and logic for smart plug state
# Temp setpoints in F
# Time setpoints in decimal hours, 24 hour time
    
    mbed_temp_upp = 67.0
    mbed_temp_low = 66.0
    mbed_on_from_time = 19.0
    mbed_on_to_time = 8.0
    
#debug conditionals
    
#print('temp under lower bound:',tstat_temp <= temp_low)
#print('time in bounds:',dectime >= on_from_time and dectime <= on_to_time)

# --Need to figure out nested conditionals, it is evaluating each of them ...
    
    if (mbed_temp <= mbed_temp_low and (dectime >= mbed_on_from_time or dectime <= mbed_on_to_time)):
    
        mbed_plug_on = True
    
    elif mbed_temp >= mbed_temp_upp or dectime < mbed_on_from_time or dectime > mbed_on_to_time:
    
        mbed_plug_on = False
   
    else:
    
        mbed_plug_on = mbed_plug_on
    
    #End logic
    
    print(mbed_plug_on)
    
#print(timestamp, tstat_name, office_temp, 'Office plug is On? ',office_plug_on)

# 6)  Call smart switch API(s) to set switch states

    mbed_plug = SmartPlug("YOUR_IP_ADDRESS")

#    print(mbed_plug.state)

    mbed_plug_status = mbed_plug.state

    if mbed_plug_status == "ON" and mbed_plug_on == False:
    
        mbed_plug.state = "OFF"
#        print("Current state: %s" % mbed_plug.state)
        mbed_plug_status = mbed_plug.state
    
    elif mbed_plug_status == "OFF" and mbed_plug_on == True:
    
        mbed_plug.state = "ON"
#        print("Current state: %s" % mbed_plug.state)
        mbed_plug_status = mbed_plug.state
    
    else:

#        print("Current state: %s" % mbed_plug.state,"no change")

        mbed_plug_status = mbed_plug_status

# 7)  Write variables, states and responses to log file

    mbed_temp_str = str(mbed_temp)
    
    l.write(timestamp)
    l.write(",")
    l.write(mbed_temp_str)
    l.write(",")
    l.write(mbed_plug_status)

    print(timestamp,mbed_temp_str,mbed_plug_status)
    
    sleep(180)

# 8) Very optional ... plot variables

In [137]:
# Test 4) calling the API for data

thermostat_url = 'https://api.ecobee.com/1/thermostat?'
payload = {'json': ('{"selection":{"selectionType":"registered","includeSensors":"true"}}')}
header = {'Content-Type': 'application/json;charset=UTF-8','Authorization': auth}


request = requests.get(thermostat_url,headers=header,params=payload)

thermostat_response = request.json()['thermostatList']

print(thermostat_response)

[{u'name': u'Home', u'lastModified': u'2017-09-23 20:55:59', u'brand': u'ecobee', u'remoteSensors': [{u'code': u'F5CK', u'name': u'M BED', u'inUse': True, u'capability': [{u'type': u'temperature', u'id': u'1', u'value': u'754'}, {u'type': u'occupancy', u'id': u'2', u'value': u'true'}], u'type': u'ecobee3_remote_sensor', u'id': u'rs:100'}, {u'capability': [{u'type': u'temperature', u'id': u'1', u'value': u'753'}, {u'type': u'humidity', u'id': u'2', u'value': u'36'}, {u'type': u'occupancy', u'id': u'3', u'value': u'true'}], u'inUse': True, u'type': u'thermostat', u'id': u'ei:0', u'name': u'Home'}], u'utcTime': u'2017-09-24 01:49:10', u'modelNumber': u'athenaSmart', u'isRegistered': True, u'thermostatRev': u'170923205559', u'identifier': u'313806613834', u'thermostatTime': u'2017-09-23 18:49:10', u'features': u'HomeKit'}]


In [56]:
# Test 5) Parsing data

tstat_string = str(thermostat_response)

tstat_list = tstat_string.split(",")

tstat_tuple = tuple(tstat_list)

#print(tstat_tuple)

#i = 0

#while i <= 33:
#
#    print(i, tstat_tuple[i])
#    i = i+1
#
# get the values from the tuple

tstat_name = tstat_tuple[26][12:-3]
print(tstat_name)
office_temp = float(tstat_tuple[16][13:-2])/10
print(office_temp)
mbed_temp = float(tstat_tuple[8][13:-2])/10
print(mbed_temp)

Home
78.4
76.0


In [75]:
# Test 6) Logic

timestamp = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())

dectime = float(timestamp[11:13])+float(timestamp[14:16])/60
    
# Setpoints and logic for smart plug state
# Temp setpoints in F
# Time setpoints in decimal hours, 24 hour time
    
office_temp_upp = 68.0
office_temp_low = 65.0
office_on_from_time = 20.5
office_on_to_time = 8.5
    
#debug conditionals
    
#print('temp under lower bound:',tstat_temp <= temp_low)
#print('time in bounds:',dectime >= on_from_time and dectime <= on_to_time)
    
if office_temp <= office_temp_low and dectime >= office_on_from_time and dectime <= office_on_to_time:
    office_plug_on = True
    office_plug_state = 1
elif office_temp >= office_temp_upp or dectime < office_on_from_time or dectime > office_on_to_time:
    office_plug_on = False
    office_plug_state = 2
else:
    office_plug_on = office_plug_on
    office_plugstate = 3
    
    #End logic
    
print(timestamp, tstat_name, office_temp, 'Office plug is On? ',office_plug_on)

('2017-09-23 15:57:29', 'Home', 78.2, 'Office plug is On? ', False)


In [110]:
# Test 7) Wemo (not completed, using TC-Link instead)

from ouimeaux.environment import Environment

def on_switch(switch):
    print "Switch found!", switch.name

def on_motion(motion):
    print "Motion found!", motion.name

env = Environment(on_switch, on_motion)

env.start()
env.discover(seconds=3)

Switch found! bedroom


In [203]:
# Test 7) tp-link!

from pyHS100 import SmartPlug
from pprint import pformat as pf

plug = SmartPlug("YOUR_IP_ADDRESS")

# Initial conditions for test only

plug_on = False

plug_status = plug.state

#  Logic

if plug_status == "ON" and plug_on == False:
    
    plug.state = "OFF"
    plug_status = plug.state
    
elif plug_status == "OFF" and plug_on == True:
    
    plug.state = "ON"
    plug_status = plug.state
    
else:

    plug_status = plug.state
    
print(plug_status)



OFF


In [154]:
# Test 8) Log File

l=open("log.txt","a+")
i=0
mbed_temp = 10.0


while i <= 3:

    mbed_temp_str = str(mbed_temp)
    
    l.write(timestamp)
    l.write(",")
    l.write(mbed_temp_str)
    l.write(",")
    l.write(mbed_plug.state)
    sleep(10)
    
    i= i+1
    mbed_temp = mbed_temp+0.5
    
    print(timestamp,mbed_temp_str,mbed_plug.state)
    

print("done")


('2017-09-23 18:55:36', '10.0', u'OFF')
('2017-09-23 18:55:36', '10.5', u'OFF')


SmartPlugException: 

In [196]:
import pyecobee

token_response = ecobee_service.refresh_tokens()

thermostat_summary_response = ecobee_service.request_thermostats_summary(selection=Selection(
        selection_type=SelectionType.REGISTERED.value,
        selection_match='',
        include_equipment_status=True))

print(thermostat_summary_response)


[16/Sep/2017 09:59:04] ERROR - EcobeeAuthorizationException raised:
Traceback (most recent call last):
  File "/Users/lenoxpod/anaconda/lib/python3.5/site-packages/pyecobee/service.py", line 155, in __process_http_response
    error_response.error_uri)
pyecobee.exceptions.EcobeeAuthorizationException: ecobee authorization error encountered for URL => https://api.ecobee.com/token?grant_type=refresh_token&client_id=tbDjHwLb4cDaKYs4OAat3oImu37e9niu
HTTP error code => 400
Error type => invalid_request
Error description => The request is malformed. Check parameters.
Error URI => https://tools.ietf.org/html/rfc6749#section-5.2


EcobeeAuthorizationException: ecobee authorization error encountered for URL => https://api.ecobee.com/token?grant_type=refresh_token&client_id=tbDjHwLb4cDaKYs4OAat3oImu37e9niu
HTTP error code => 400
Error type => invalid_request
Error description => The request is malformed. Check parameters.
Error URI => https://tools.ietf.org/html/rfc6749#section-5.2

In [192]:
token_response = ecobee_service.refresh_tokens()
print(token_response)

[16/Sep/2017 09:53:51] ERROR - EcobeeAuthorizationException raised:
Traceback (most recent call last):
  File "/Users/lenoxpod/anaconda/lib/python3.5/site-packages/pyecobee/service.py", line 155, in __process_http_response
    error_response.error_uri)
pyecobee.exceptions.EcobeeAuthorizationException: ecobee authorization error encountered for URL => https://api.ecobee.com/token?grant_type=refresh_token&client_id=tbDjHwLb4cDaKYs4OAat3oImu37e9niu
HTTP error code => 400
Error type => invalid_request
Error description => The request is malformed. Check parameters.
Error URI => https://tools.ietf.org/html/rfc6749#section-5.2


EcobeeAuthorizationException: ecobee authorization error encountered for URL => https://api.ecobee.com/token?grant_type=refresh_token&client_id=tbDjHwLb4cDaKYs4OAat3oImu37e9niu
HTTP error code => 400
Error type => invalid_request
Error description => The request is malformed. Check parameters.
Error URI => https://tools.ietf.org/html/rfc6749#section-5.2

In [209]:
# test file writing

t = open("test","a+")

testlist = []

i=0

while i <= 3:
    
    eye = str(i)
    
    testlist.append(eye)
    
    i = i+1
    
t.write(testlist)
t.close
    
end


TypeError: expected a string or other character buffer object

In [182]:
from time import sleep
import datetime

#initialize plug states before loop

office_plug_on = False 
mbed_plug_on = False 
orm_plug_on = False 
lrm_plug_on = False 

while True:
        
# Full payload here for reference, not used  
# selection = Selection(selection_type=SelectionType.REGISTERED.value, selection_match='', include_alerts=True,
#                     include_device=True, include_electricity=False, include_equipment_status=False,
#                    include_events=False, include_extended_runtime=False, include_house_details=False,
#                      include_location=False, include_management=False, include_notification_settings=False,
#                      include_oem_cfg=False, include_privacy=False, include_program=False, include_reminders=False,
#                      include_runtime=False, include_security_settings=False, include_sensors=True,
#                      include_settings=False, include_technician=False, include_utility=False, include_version=False,
#                      include_weather=False)

    #selection of parameters
    
    selection = Selection(selection_type=SelectionType.REGISTERED.value, selection_match='',include_sensors=True)

    #Returns the thermostat_response object including the temperature readings

    thermostat_response = ecobee_service.request_thermostats(selection)

    #convert this response to a string, and then a tuple

    tstat_string = str(thermostat_response)

    tstat_list = tstat_string.split(",")

    tstat_tuple = tuple(tstat_list,)

    #uncomment and use this to determine location of new values
    # e.g. remote sensor readings
    
    x = 0
    
    while x <= 100:
   
        print(x, tstat_tuple[x]) 
    
        x = x+1
    
    # get the values from the tuple

    tstat_name = tstat_tuple[47][7:-1]

    tstat_temp_string = tstat_tuple[53][8:-2]

    #Convert temp string to an number

    tstat_temp = float(tstat_temp_string)/10
    
    #timestamp and time for logic

    timestamp = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())

    dectime = float(timestamp[11:13])+float(timestamp[14:16])/60
    
    # Setpoints and logic for smart plug state
    # Temp setpoints in F
    # Time setpoints in decimal hours, 24 hour time
    
    office_temp_upp = 68.0
    office_temp_low = 65.0
    office_on_from_time = 19.0
    office_on_to_time = 8.0
    
    #debug conditionals
    
    #print('temp under lower bound:',tstat_temp <= temp_low)
    #print('time in bounds:',dectime >= on_from_time and dectime <= on_to_time)
    
    if tstat_temp <= office_temp_low and dectime >= office_on_from_time and dectime <= office_on_to_time:
        office_plug_on = True
        office_plug_state = 1
    elif tstat_temp >= office_temp_upp or dectime < office_on_from_time or dectime > office_on_to_time:
        office_plug_on = False
        office_plug_state = 2
    else:
        office_plug_on = office_plug_on
        office_plugstate = 3
    
    #End logic
    
    print(timestamp, tstat_name, tstat_temp, 'Office plug is On? ',office_plug_on)
    
    sleep(300) 

    
    token_response = ecobee_service.refresh_tokens() #refresh token

    #Next step ... Do similar to above for sensors
    #Exception handling to exit the loop & error message
    #logging
    #Link to smart plug
    #plotly?


AttributeError: 'ConnectTimeout' object has no attribute 'message'

In [59]:
    # logic simulator -- test before using above
    i = 0
    j = 0
    tstat_temp = 65.0
    dectime = 0
    plug_on = True
    temp_upp = 68.0
    temp_low = 65.0
    on_from_time = 1
    on_to_time = 2
    
    for i in range (0,4):
        for j in range (0,6):
     
            if tstat_temp <= temp_low and dectime >= on_from_time and dectime <= on_to_time:
                plug_on = True
            elif tstat_temp >= temp_upp or dectime < on_from_time or dectime > on_to_time:
                plug_on = False
            else:
                plug_on = plug_on
        
            print(dectime, tstat_temp, plug_on)
            tstat_temp = tstat_temp + 1  
            j = j+1
        dectime = dectime + 1
        tstat_temp = 65
        i = i+1
    print('_________________________________')    
    tstat_temp = 70
    dectime = 0
    for i in range (0,4):
        for j in range (0,6):
     
            if tstat_temp <= temp_low and dectime >= on_from_time and dectime <= on_to_time:
                plug_on = True
            elif tstat_temp >= temp_upp or dectime < on_from_time or dectime > on_to_time:
                plug_on = False
            else:
                plug_on = plug_on
        
            print(dectime, tstat_temp, plug_on)
            tstat_temp = tstat_temp - 1  
            j = j+1
        dectime = dectime + 1
        tstat_temp = 70
        i = i+1

(0, 65.0, False)
(0, 66.0, False)
(0, 67.0, False)
(0, 68.0, False)
(0, 69.0, False)
(0, 70.0, False)
(1, 65, True)
(1, 66, True)
(1, 67, True)
(1, 68, False)
(1, 69, False)
(1, 70, False)
(2, 65, True)
(2, 66, True)
(2, 67, True)
(2, 68, False)
(2, 69, False)
(2, 70, False)
(3, 65, False)
(3, 66, False)
(3, 67, False)
(3, 68, False)
(3, 69, False)
(3, 70, False)
_________________________________
(0, 70, False)
(0, 69, False)
(0, 68, False)
(0, 67, False)
(0, 66, False)
(0, 65, False)
(1, 70, False)
(1, 69, False)
(1, 68, False)
(1, 67, False)
(1, 66, False)
(1, 65, True)
(2, 70, False)
(2, 69, False)
(2, 68, False)
(2, 67, False)
(2, 66, False)
(2, 65, True)
(3, 70, False)
(3, 69, False)
(3, 68, False)
(3, 67, False)
(3, 66, False)
(3, 65, False)


In [160]:
timestamp = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())

hour = timestamp[12:13]
minute = timestamp[14:16]
decminute = float(minute)/60

print(hour, minute, decminute)

dectime = float(timestamp[12:13])+float(timestamp[14:16])/60 #something is screwy with this math

print(timestamp, dectime)

9 39 0.65
2017-09-10 09:39:20 9.65


In [78]:
#test file write / read for tokens

#Write tokens to file

f=open("tokens.txt","w+")
f.write(access_token)
f.write(",")
f.write(token_type)
f.write(",")
f.write(refresh_token)
f.close

access_token="none"
token_type="none"
refresh_token="none"

f=open("tokens.txt","r")

token_str = str(f.read())

f.close

token_list = token_str.split(",")

access_token = token_list[0]
token_type = token_list[1]
refresh_token = token_list[2]

print("access token:", access_token)
print("token type:", token_type)
print("refresh token:", refresh_token)

('access token:', '7Qnfp0tLpj4o3WZJhTfSrUDOuoQD21Dl')
('token type:', '3b672Qx2B4gk3AExIlfnPMKGxkm71ngS')
('refresh token:', '3b672Qx2B4gk3AExIlfnPMKGxkm71ngS')
